In [1]:
import os
import time

import mido
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt

In [2]:
def getData(data_folder):
    """
    Walk through a folder, modifing each mid to fit in 128 ticks per beat and be only a piano channel
    :param data_folder: the base data folder used in the model, still nonrepresentative
    :return: it saves a midfile in piano format and other in paino+128tpb
    """
    counter = 0 
    for dirpath, _, filenames in os.walk(data_folder):
        for File in filenames:
            path = os.path.join(dirpath, File)
            mid_copy = mido.MidiFile()
            
            try:
                mid = mido.MidiFile(path)
                tpb = mid.ticks_per_beat
                duration = int(mid.length)

                mid_copy.type = 1
                mid_copy.ticks_per_beat = tpb

                if 10 < duration < 600:
                    track = mido.merge_tracks(mid.tracks)

                    for msg in list(track):
                        if not msg.is_meta and msg.channel != 0 and msg.type == "note_on":
                            msg.velocity = 0
                            
                mid_copy.tracks.append(track)
                mid_copy.save(f"OnlyPiano/{File}")
            
                mid_copy.ticks_per_beat = 128
                mid_copy.save(f"OnlyPiano_TPB128/{File}") 
                
            except:
                print(path)
                continue
            
            counter+=1
            if counter %250 ==0:
                print(counter)

In [3]:
start_time = time.time()
getData('TestDataset')
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.5220174789428711 seconds ---


In [4]:
def get_info(data_folder):
    counter = 0
    for dirpath, _, filenames in os.walk(data_folder):
        for File in filenames:
            path = os.path.join(dirpath, File)

            try:
                mid = mido.MidiFile(path)
                duration = mid.length
                track = mido.merge_tracks(mid.tracks)
                delta_time_ticks = 0
            
                for msg in list(track):
                    delta_time_ticks += msg.time
#                     if not msg.is_meta and msg.channel ==0:
#                         print(msg)
                beat_num = int(delta_time_ticks/128)
                num_compasso = int(beat_num/4)
                print(duration)
                print(delta_time_ticks)
                print(beat_num)
                print(num_compasso)
                print(int(num_compasso - 3))
                
            except:
                print(path)
                continue
                
            return mid  

In [5]:
mid = get_info('OnlyPiano_TPB128')

63.285651000000584
14176
110
27
24


In [6]:
num_notes = 128
samples_per_measure = 128

def midi_to_pianoRoll(mid):
    mid.ticks_per_beat = 128
    track = mido.merge_tracks(mid.tracks)
    
    abs_time = 0
    counter = 1
    samples = []
    
    for msg in track:
        abs_time += msg.time
        [samples.append([0]*128) for i in range(msg.time)]
        if not msg.is_meta and msg.channel != 0:
            continue
        
#         if msg.type == "note_on":
#             note = msg.note
#             for i in range(abs_time - msg.time,abs_time):
#                 samples[i][note] = 0
#             print(f"Note -> {note}")
#             print(f"ON -> {msg.time}")

        if msg.type == "note_off":
            note = msg.note
            for i in range(abs_time - msg.time,abs_time):
                samples[i][note] = 1

#             print(f"OFF -> {msg.time}")
#             print(f"total ticks = {abs_time}\n")
            
    return samples

In [7]:
start_time = time.time()
samples = midi_to_pianoRoll(mid)
print("--- %s seconds ---" % (time.time() - start_time))
samples = np.asarray(samples)

--- 0.3512256145477295 seconds ---


In [43]:
def samples_to_midi(samples, fname):
    mid = mido.MidiFile()
    track = mido.MidiTrack()
    mid.tracks.append(track)
    mid.ticks_per_beat = 128
    
    abs_time = 0
    last_time = 0
    
    for tick in range(samples.shape[0]):
        abs_time += 1

        for note in range(samples.shape[1]):
            if samples[tick,note] == 1 and (tick == 0 or samples[tick-1,note] == 0):
                delta_time = abs_time - last_time
                delta_time -= 16
                if delta_time < 0:
                    delta_time = 0

                track.append(mido.Message('note_on', note=note, channel = 0, time= abs(delta_time)))
                
                last_time = abs_time

            if samples[tick,note] == 1 and (tick == samples.shape[0]-1 or samples[tick+1,note] == 0):
                track.append(mido.Message('note_off', note=note, channel = 0, time= 16))
#     return mid
    mid.save(fname + ".mid")
    

In [47]:
# plt.imshow(samples.T[:,0:512])
# plt.show()

In [45]:
start_time = time.time()
samples_to_midi(samples, 'teste')
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.928210973739624 seconds ---


In [44]:
track = mido.merge_tracks(teste.tracks)

for msg in track:
    if not msg.is_meta:
        print(msg)

note_on channel=0 note=76 velocity=64 time=0
note_off channel=0 note=76 velocity=64 time=10
note_on channel=0 note=76 velocity=64 time=22
note_off channel=0 note=76 velocity=64 time=10
note_on channel=0 note=76 velocity=64 time=30
note_off channel=0 note=76 velocity=64 time=10
note_on channel=0 note=72 velocity=64 time=38
note_off channel=0 note=72 velocity=64 time=10
note_on channel=0 note=76 velocity=64 time=14
note_off channel=0 note=76 velocity=64 time=10
note_on channel=0 note=79 velocity=64 time=38
note_off channel=0 note=79 velocity=64 time=10
note_on channel=0 note=72 velocity=64 time=182
note_off channel=0 note=72 velocity=64 time=10
note_on channel=0 note=67 velocity=64 time=70
note_off channel=0 note=67 velocity=64 time=10
note_on channel=0 note=64 velocity=64 time=54
note_off channel=0 note=64 velocity=64 time=10
note_on channel=0 note=69 velocity=64 time=62
note_off channel=0 note=69 velocity=64 time=10
note_on channel=0 note=71 velocity=64 time=46
note_off channel=0 note=

In [ ]:
# def split_mid(sample):
#     splited = []
#     sample = sample.T
    
#     for i in range(0,len(sample),512):
#         splited_sample = sample[i+2048:i]
#         splited.append(splited_sample)
        
#         print(np.shape(splited_sample))
#         samples_to_midi(splited_sample.T, f"Splited/{int(i/512)}")
        
# #     return splited 

In [ ]:
# list_splited = split_mid(samples)